[View in Colaboratory](https://colab.research.google.com/github/DerekChuang/tensorflow_practice/blob/master/numpy&pandas_input.ipynb)

In [0]:
import tensorflow as tf
import numpy as np
import shutil

In [0]:
shutil.rmtree("outdir", ignore_errors = True) # start fresh each time


In [0]:
sqft  = np.array([ 1000,    2000,    3000,    1000,  2000,  3000])
ptype = np.array(["house", "house", "house", "apt", "apt", "apt"])
price = np.array([ 500,     1000,    1500,    700,   1300,  1900])


# numpy_input_fn
tf.estimator.inputs.numpy_input_fn(
    x,
    y=None,
    batch_size=128,
    num_epochs=1,
    shuffle=None,
    queue_capacity=1000,
    num_threads=1
  )
  
  
  
  x: numpy array object or dict of numpy array objects. If an array, the array will be treated as a single feature.
y: numpy array object or dict of numpy array object. None if absent.

In [0]:
def nptrain_input_fn(sqft, ptype, price):  # a numpy array
   return tf.estimator.inputs.numpy_input_fn(
       x = {"sq_footage": sqft, "type": ptype},
       y = price,
       num_epochs=1000,
       batch_size=3,
       shuffle=True
   )
  
  
  
  
  
  
  

feat, lab = nptrain_input_fn(sqft, ptype, price)()

Instructions for updating:
To construct input pipelines, use the `tf.data` module.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.


In [0]:
featcols = [
    tf.feature_column.numeric_column("sq_footage"),
    tf.feature_column.categorical_column_with_vocabulary_list("type", ["house", "apt"])
]

In [0]:
model = tf.estimator.LinearRegressor(featcols, "outdir")


INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'outdir', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fae76a21cf8>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [0]:

# try geting data from the input function by hand
print(feat)
with tf.Session() as sess:
    tf.train.start_queue_runners(coord=tf.train.Coordinator())
    a = sess.run(feat["sq_footage"])
    print(a)
    a = sess.run(feat["sq_footage"])
    print(a)

{'sq_footage': <tf.Tensor 'random_shuffle_queue_DequeueUpTo:1' shape=(?,) dtype=int64>, 'type': <tf.Tensor 'random_shuffle_queue_DequeueUpTo:2' shape=(?,) dtype=string>}
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
[3000 2000 2000]
[1000 3000 3000]


In [0]:
#train model
model.train(nptrain_input_fn(sqft, ptype, price))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into outdir/model.ckpt.
INFO:tensorflow:loss = 7710000.0, step = 1
INFO:tensorflow:global_step/sec: 517.015
INFO:tensorflow:loss = 98971.27, step = 101 (0.195 sec)
INFO:tensorflow:global_step/sec: 667.555
INFO:tensorflow:loss = 100924.06, step = 201 (0.153 sec)
INFO:tensorflow:global_step/sec: 431.187
INFO:tensorflow:loss = 37713.87, step = 301 (0.227 sec)
INFO:tensorflow:global_step/sec: 719.621
INFO:tensorflow:loss = 67504.41, step = 401 (0.140 sec)
INFO:tensorflow:global_step/sec: 737.778
INFO:tensorflow:loss = 82032.26, step = 501 (0.135 sec)
INFO:tensorflow:global_step/sec: 706.435
INFO:tensorflow:loss = 52000.266, step = 601 (0.141 sec)
INFO:tensorflow:global_step/sec: 718.095
INFO:tensorflow:loss = 147094.

In [0]:
def predict_input_fn():
    features = {"sq_footage": [1500, 1500, 2500, 2500],
                "type": ["house", "apt", "house", "apt"]}
    return features

predictions = model.predict( input_fn = predict_input_fn)

print(next(predictions))
print(next(predictions))
print(next(predictions))
print(next(predictions))
print(next(predictions))
print(next(predictions))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from outdir/model.ckpt-2000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
{'predictions': array([857.77325], dtype=float32)}
{'predictions': array([879.367], dtype=float32)}
{'predictions': array([1436.0916], dtype=float32)}
{'predictions': array([1457.6853], dtype=float32)}
{'predictions': array([857.77325], dtype=float32)}
{'predictions': array([879.367], dtype=float32)}


# pandas_input_fn

In [0]:
shutil.rmtree("outdir", ignore_errors = True) # start fresh each time


In [39]:
import pandas as pd



#建構一個 dictionary
dict = {
    
    "sqft" : [ 1000,    2000,    3000,    1000,  2000,  3000],
    "ptype" : ["house", "house", "house", "apt", "apt", "apt"],
    "price" : [ 500,     1000,    1500,    700,   1300,  1900]

}


df = pd.DataFrame(dict)
df

,price,ptype,sqft
0,500,house,1000
1,1000,house,2000
2,1500,house,3000
3,700,apt,1000
4,1300,apt,2000
5,1900,apt,3000


In [0]:
def pdtrain_input_fn(df):  # a pandas df
   return tf.estimator.inputs.pandas_input_fn(
       x = df,
       y = df['price'],
       num_epochs=1000,
       batch_size=3,
       shuffle=True
   )
  

In [41]:
featcols = [
    tf.feature_column.numeric_column("sqft"),
    tf.feature_column.categorical_column_with_vocabulary_list("ptype", ["house", "apt"])
]

model = tf.estimator.LinearRegressor(featcols, "outdir")


INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'outdir', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fae71e662b0>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [42]:
model.train(pdtrain_input_fn(df))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into outdir/model.ckpt.
INFO:tensorflow:loss = 2750000.0, step = 1
INFO:tensorflow:global_step/sec: 359.558
INFO:tensorflow:loss = 26962.484, step = 101 (0.281 sec)
INFO:tensorflow:global_step/sec: 347.602
INFO:tensorflow:loss = 58713.75, step = 201 (0.293 sec)
INFO:tensorflow:global_step/sec: 382.158
INFO:tensorflow:loss = 68190.92, step = 301 (0.257 sec)
INFO:tensorflow:global_step/sec: 412.574
INFO:tensorflow:loss = 29844.727, step = 401 (0.243 sec)
INFO:tensorflow:global_step/sec: 356.087
INFO:tensorflow:loss = 72402.46, step = 501 (0.281 sec)
INFO:tensorflow:global_step/sec: 354.932
INFO:tensorflow:loss = 71262.41, step = 601 (0.286 sec)
INFO:tensorflow:global_step/sec: 385.17
INFO:tensorflow:loss = 38679.05

In [44]:
def predict_input_fn():
    features = {"sqft": [1500, 1500, 2500, 2500],
                "ptype": ["house", "apt", "house", "apt"]}
    return features

predictions = model.predict( input_fn = predict_input_fn)

print(next(predictions))
print(next(predictions))
print(next(predictions))
print(next(predictions))
print(next(predictions))
print(next(predictions))


INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from outdir/model.ckpt-2000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
{'predictions': array([855.5049], dtype=float32)}
{'predictions': array([879.28687], dtype=float32)}
{'predictions': array([1431.9532], dtype=float32)}
{'predictions': array([1455.7352], dtype=float32)}
{'predictions': array([855.5049], dtype=float32)}
{'predictions': array([879.28687], dtype=float32)}
